# MultiLayer Perceptron

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, Flatten, Input, Dense, Dropout, ReLU
from tensorflow.keras.layers import Concatenate, Lambda
from tensorflow.keras.regularizers import l2
# Printing
import pprint
import matplotlib.pyplot as plt
import sys
sys.path.append('../source/')
sys.path.append('../scripts/')
from predictionModel import PredictionModel

In [2]:
#speeds = pd.read_pickle('data/speeds1419.pckl')
#speeds80 = speeds.groupby('data/matching_road').mean().dropna(thresh = int(0.8*len(speeds.columns)))

#counts = pd.read_pickle('data/counts1419.pckl')
#mergeResults=pd.read_pickle("../data/mergeResults.pckl")
#segmentsMeta=pd.read_pickle("../data/segmentsMeta.pckl")

updatedSpeed = pd.read_pickle("../data/updatedSpeedWithHistoricalValues1419.pckl")

In [5]:
updatedSpeed.head()

time,2018-11-13 14:00:00,2018-11-13 14:15:00,2018-11-13 14:30:00,2018-11-13 14:45:00,2018-11-13 15:00:00,2018-11-13 15:15:00,2018-11-13 15:30:00,2018-11-13 15:45:00,2018-11-13 16:00:00,2018-11-13 16:15:00,...,2018-12-11 16:30:00,2018-12-11 16:45:00,2018-12-11 17:00:00,2018-12-11 17:15:00,2018-12-11 17:30:00,2018-12-11 17:45:00,2018-12-11 18:00:00,2018-12-11 18:15:00,2018-12-11 18:30:00,2018-12-11 18:45:00
newIndex,,,,,,,,,,,,,,,,,,,,,
10275171_0,6.000000,13.500000,6.487879,12.333333,10.000000,2.500000,10.000000,3.8875,3.750000,3.875000,...,0.000000,7.500000,5.750000,2.285714,12.500000,4.000000,3.850000,1.444444,6.000000,3.000000
11027377_0,38.000000,17.000000,11.500000,25.198313,22.916667,7.000000,4.666667,13.5000,46.500000,17.828788,...,12.766667,8.000000,20.000000,35.000000,25.375000,19.500000,38.000000,4.000000,0.000000,12.500000
11027379_0,80.166667,72.750000,72.000000,75.200000,78.000000,69.833333,63.666667,73.6250,82.000000,70.285714,...,71.500000,69.000000,55.250000,72.000000,75.666667,76.777778,50.500000,89.750000,60.000000,71.125000
111437857_0,84.707242,89.400000,94.000000,76.333333,87.500000,74.750000,95.000000,77.5000,44.500000,19.000000,...,6.800000,24.625000,8.333333,15.833333,15.000000,48.500000,86.000000,70.500000,55.000000,62.476190
111848631_0,102.746479,102.333333,101.183099,100.279070,101.642857,98.578313,109.395349,111.0000,109.882353,109.030303,...,108.959184,107.123457,106.111111,109.049180,112.061224,113.850575,113.764706,117.500000,116.323529,109.552632


In [3]:
input_lag, output_lag, sequence_length, valid_split =  1, 1, 20, 0.6

data_model = PredictionModel(updatedSpeed, input_lag, output_lag, sequence_length, valid_split= valid_split)
data_model.preprocessData()
x_train, y_train, x_test, y_test = data_model.trainSplit()

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (148, 748)
y_train shape: (148, 748)
x_test shape: (99, 748)
y_test shape: (99, 748)


In [41]:
def createMlpModel(loss='MSE',optimizer='Adam',nlayers=1, neuronsPerLayer=64,relu_output=False, nSegments=748):
    
    model = Sequential()
    model.add(Dense(neuronsPerLayer, input_dim = nSegments, name='input_layer'))
    
    for i in range(nlayers-1):    
        model.add(Dense(neuronsPerLayer))
        
    model.add(Dense(nSegments, name='out_layer'))
    if relu_output :
        model.add(ReLU())
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [42]:
model = createMlpModel()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Dense)          (None, 64)                47936     
_________________________________________________________________
out_layer (Dense)            (None, 748)               48620     
Total params: 96,556
Trainable params: 96,556
Non-trainable params: 0
_________________________________________________________________


In [11]:
def createMlpModel(loss='MSE',optimizer='Adam', nlayers=1, neuronsPerLayer=64, relu_output=False, nSegments=748):
    
    inputs = Input(shape=(nSegments,), name='input')
    x = Flatten(name='flatten')(inputs)
    
    for i in range(nlayers):    
        x = Dense(neuronsPerLayer/(i+1))(x)
        
    if relu_output :
        predictions = Dense(nSegments, activation='relu', name='out_layer')(x)
    else:
        predictions = Dense(nSegments, name='out_layer')(x)
        
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [13]:
data_model.model = createMlpModel(nlayers=1, neuronsPerLayer=64, relu_output=False)
data_model.model.summary()
data_model.model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs=40)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 748)               0         
_________________________________________________________________
flatten (Flatten)            (None, 748)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                47936     
_________________________________________________________________
out_layer (Dense)            (None, 748)               48620     
Total params: 96,556
Trainable params: 96,556
Non-trainable params: 0
_________________________________________________________________
Train on 148 samples, validate on 99 samples
Epoch 1/40
148/148 [==============================] - 0s 2ms/step - loss: 4338.1596 - val_loss: 3259.0622
Epoch 2/40
148/148 [==============================] - 0s 199us/step - loss: 2598.4082 - val_loss: 1598.6934
Epoch 3/40


In [14]:
yDF=data_model.restorePredictionsAsDF(data_model.y)
predDF = data_model.restorePredictionsAsDF(data_model.predict('full'))

colors = ((np.abs(yDF - predDF)+1)/(yDF+1)).clip(upper=1)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 2 arrays: [array([[  6.        ,  38.        ,  80.16666667, ..., 106.34375   ,
         23.61507937,  41.33333333],
       [ 13.5       ,  17.        ,  72.75      , ..., 106.25806452,
         20.5       ,  4...

In [ ]:
def plotPredHeatMap(yDF,predDF,cost=None):
    if cost :
        colors= cost(yDF,predDF)
    else :
        colors = ((np.abs(yDF - predDF)+1)/(yDF+1)).clip(upper=1)
    
    plt.figure(figsize=(18,12))
    plt.imshow(colors,aspect='auto')
    plt.colorbar()
    
plotPredHeatMap(yDF,predDF)

In [34]:
tf.keras.backend.clear_session()